# Einführung in die Parallelisierung

### Motivation

In [1]:
## Beispiel 0: Approximation von Pi mit nur einem Prozess
using Distributed

# Teste wie viele Prozesse arbeiten
print("Number of Cores: ", nprocs()," and Number of Workers: ", nworkers(), "    ---    ")

# define a function that counts the number of points falling inside the circle
@everywhere function points_inside_circle(n)
    n_in = 0
    for i = 1:n
        x, y = rand(), rand()
        n_in += (x*x + y*y) <= 1
    end
    return n_in
end

# define a function wrapper that computes the approximation of pi in parallel
@everywhere function pi_p(n)    
    p = nworkers()
    n_in = @distributed (+) for i = 1:p
        points_inside_circle(n/p)
    end
    return 4 * n_in / n
end


@time pi_p(1_000_000_000)


Number of Cores: 1 and Number of Workers: 1    ---     11.859163 seconds (385.42 k allocations: 19.520 MiB, 0.09% gc time)


3.141582132

### Erste Schritte

### Beispiele

In [2]:
## Beispiel 1: Füge drei neue Prozesse hinzu 
using Distributed
addprocs(3)
print("Number of Cores: ", nprocs()," and Number of Workers: ", nworkers(), "    ---    ")
# Somit haben wir einen Leader (immer ID 1) und n Worker verteilt auf n+1 Prozesse

Number of Cores: 4 and Number of Workers: 3    ---    

In [3]:
## Beispiel 2: Rufe Informationen über die Prozesse ab
for i in workers()
    id, pid, host = fetch(@spawnat i (myid(), getpid(), gethostname()))
    print("| I have the ID: ", id, " with PID: ", pid, " and HOST: ", host, " |")
end

| I have the ID: 2 with PID: 17760 and HOST: MacBook-3.local || I have the ID: 3 with PID: 17761 and HOST: MacBook-3.local || I have the ID: 4 with PID: 17762 and HOST: MacBook-3.local |

### Prozesse ausführen

Wichtige Funktionen: 
* $remotecall()$ - Aufgabe von einem bestimmten Prozess ausführen lassen. (Asynchroner Aufruf, kein Blocking, Return sofort)
* $fetch()$ - Ergebnis abholen. (Julia verwendet eine Future-Referenz. Diese ist sofort verfügbar, das Ergebnis muss aber abgholt werden.)
* $remotecall$_$fetch()$ - Synchroner Aufruf inkl. Abholung des Ergebnisses. (Kombiniert $remotecal()$ und $fetch()$.)
* $@spawn$ - Aufgaben an andere Prozesse senden
* $@spawnat$ - Aufgabe an einen bestimmten Prozess senden

### Beispiele

In [4]:
## Beispiel 3: Prozess mit ID 2 soll 1 + 3 rechnen

result = remotecall(+, 2, 1, 3) 

fetch(result) # Ergebnis muss abgeholt werden

4

In [5]:
## Beispiel 4: Prozess 2 nutzt Ergebnis von Prozess 4

result = remotecall(rand, 4, 3, 4) # Prozess 4 erzeugt eine 3x4 Zufallsmatrix
s = @spawnat 2 1 .+ fetch(result)  # Prozess 2 benutzt das Ergebnis von 4 und hängt 1. davor
fetch(s)

3×4 Array{Float64,2}:
 1.08486  1.45546  1.89077  1.21451
 1.22418  1.51434  1.64637  1.0909
 1.9326   1.66997  1.31625  1.95114

In [6]:
## Beispiel 5: Approximation von Pi mit mehreren Prozessen
# Mit @distributed wird die for-loop in der zweiten Funktion verteilt ausgeführt.
using Distributed
# define a function that counts the number of points falling inside the circle
@everywhere function points_inside_circle(n)
    n_in = 0
    for i = 1:n
        x, y = rand(), rand()
        n_in += (x*x + y*y) <= 1
    end
    return n_in
end

# define a function wrapper that computes the approximation of pi in parallel
@everywhere function pi_p(n)    
    p = nworkers()
    n_in = @distributed (+) for i = 1:p 
        points_inside_circle(n/p)
    end
    return 4 * n_in / n
end


@time pi_p(1_000_000_000)


  6.381167 seconds (83.80 k allocations: 4.139 MiB)


3.141593112

Wie man hier sehen  kann, halbiert sich die Berechnungszeit fast, indem man verteilt auf vier Prozesse rechnet.

# Einführung in das Paket Distributed Arrays
Große Berechnungen sind meist aus großen Arrays aufgebaut.
In diesem Fall macht es Sinn, die Arrays unter verschiedenen Prozessen aufzuteilen.
Indem die Speicherkapazitäten von verschiedenen Maschinen genutzt werden können, können so unter anderem Arrays berechnet werden, die sonst zu groß wären.
Jeder Prozess kann den ihm zugewiesenen Teil des Arrays lesen und überschreiben. Gleichzeitig kann jeder Prozess das vollständige Array lesen (read-only).

In Julia werden Distributed Arrays durch den "DArray"-Typ beschrieben. 
* Elementtyp und Dimensionen sind wie bei normalen Arrays
* Enthaltene Daten werden durch eine Aufteilung der Indexmenge in eine bestimmte Anzahl von Blöcken in jeder Dimension aufgeteilt

Vorteil:
* When dividing data among a large number of processes, one often sees diminishing returns in performance. Placing DArrays on a subset of processes allows multiple DArray computations to happen at once, with a higher ratio of work to communication on each process.

In [9]:
@everywhere using DistributedArrays

### Beispiele für Distributed Arrays:
* Julia übernimmt hier die gesamte Arbeit: Die einzelnen Matrixelemente sind auf die verschiedenen Prozesse verteilt und auch initialisiert.

### Für mehr Kontrolle:
* Spezifizifikation eines bestimmten Prozesses für eine Aufgabe
* Festlegung, wie die Daten aufgeteilt werden sollen

Beispiel:

In [10]:
#= 
Second argument: Array should be created on the first three workers.
Third argument: Specify a distribution; the nth element of this array specifies
how many pieces dimension n should be divided into.
HERE: The first dimension will not be divided, the second dimension will be divided into 4 pieces.
Therefore each local chunk will be of size (100,25).
=#
dzeros((100,100), workers()[1:3], [1,3])

100×100 DArray{Float64,2,Array{Float64,2}}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0 

## Hilfreiche Funktionen:

Note: Indexing a DArray (square brackets) with ranges of indices always creates a SubArray, not copying any data.

# Initialisierung

* $init$ - vom Programmierer bereitzustellen; bekommt ein Tupel von Index-Bereichen übergeben, für die das Array dann innerhalb der Init-Funktion initialisiert wird; Init-Funktion initialisiert also lokale Bereiche des Distributed Arrays
* $dims$ - Dimension des Distributed Arrays 
* $procs$ - Optional: Spezifiziert einen Vektor von Prozess IDs, die genutzt werden sollen
* $dist$ - Optional: Integer Vektor, der angibt wie der Distributed Array in jeder Dimension unter den Prozessen aufgeteilt werden soll

### Beispiel: Verteilte Zufallsmatrix

In [11]:
@everywhere using DistributedArrays
# ------------------------------------
#=Erzeuge eine verteilte Zufallsmatrix und gib den Teil zurück, für den Prozess 2 verantwortlich ist.
Mehrmaliges Hintereinanderausführen führt zu einer immer feineren, automatischen Aufteilung der Matrix,
da mehr Prozesse hinzugefügt werden.=#

d = drand(10,10);
r = @spawnat 2 begin
    localpart(d)
end
fetch(r)

10×4 Array{Float64,2}:
 0.375114   0.515236   0.544062  0.807627
 0.913663   0.382798   0.974972  0.919571
 0.223225   0.170889   0.637931  0.136496
 0.886466   0.524885   0.5522    0.838163
 0.156507   0.850004   0.910517  0.0415769
 0.504012   0.30831    0.765573  0.956131
 0.420482   0.918328   0.523361  0.0341088
 0.9587     0.981585   0.630585  0.599726
 0.0495685  0.0978773  0.82137   0.422337
 0.918235   0.248038   0.903477  0.343083

Das Praktische: 
Julia versteckt die gesamte Kommunikation vor dem Benutzer. Will man z.B. die Summe aller Elemente der Matrix berechnen, so ruft man einfach $sum(d)$ auf. Das liefert direkt das Ergebnis. Julia summiert dabei mit jedem einzelnen Prozess alle Elemente des Teilarray auf. Anschließend werden dann alle Teilergebnisse zur Gesamtsumme addiert.

In [12]:
#=Erzeuge eine verteilte Zufallsmatrix.=#

d = drand(10,10);

@time sum(d)

  1.187323 seconds (1.59 M allocations: 91.722 MiB, 1.62% gc time)


52.68024169298725

### Beispiel: Julia mit Julia (funktioniert hier leider noch nicht)

http://mathemartician.blogspot.com/2012/07/julia-set-in-julia.html

zum Nachlesen!

In [21]:
#DistributedArrays, WIDTH, HEIGHT und MAXITER sind global
@everywhere WIDTH = 1000
@everywhere HEIGHT = 500
@everywhere MAXITER = 100

#Images ist lokal
using Images

# Julia-Funktion
@everywhere function julia(z, maxiter::Int64)
    c = -0.8 + 0.156im
    for n = 1:maxiter
        if abs(z) > 2.0
            return n-1
        end
        z = z^2 + c
    end
    return maxiter
end

# Init-Funktion zum Anlegen des Arrays
@everywhere function parJuliaInit(I)
    e = (size(I[1], 1), size(I[2], 1))
    m = Array(Int, e)
    xMin=I[2][1]
    yMin=I[1][1]
    
    for x = I[2], y = I[1]
        c = complex((x - WIDTH/2) / (HEIGHT/2), (y - HEIGHT/2) / (HEIGHT/2))
        m[y - yMin + 1, x - xMin + 1] = julia(c, MAXITER)
    end
    return m
end

# Distributed Array anlegen
Dm = DArray(parJuliaInit, (HEIGHT, WIDTH))

# Distributed Array auf den lokalen Prozess holen
#m = convert(Array, Dm)

#Bild als PNG speichern
#imwrite(grayim(transpose(m)/(1.0*MAXITER)),"juliaset.png")

CompositeException: TaskFailedException:
On worker 2:
TypeError: in typeassert, expected Int64, got Type{Int64}
parJuliaInit at ./In[21]:24
#construct_localparts#5 at /Users/alfonshilmer/.julia/packages/DistributedArrays/UZMDF/src/darray.jl:118
construct_localparts at /Users/alfonshilmer/.julia/packages/DistributedArrays/UZMDF/src/darray.jl:118
#104 at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.4/Distributed/src/process_messages.jl:294
run_work_thunk at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.4/Distributed/src/process_messages.jl:79
macro expansion at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.4/Distributed/src/process_messages.jl:294 [inlined]
#103 at ./task.jl:358
Stacktrace:
 [1] remotecall_fetch(::Function, ::Distributed.Worker, ::Function, ::Vararg{Any,N} where N; kwargs::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at /Users/julia/buildbot/worker/packa